In [309]:
!pip3 install pandas plotly matplotlib kaleido nbformat statsmodels

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import plotly.express as plx
import matplotlib.pyplot as plt
import kaleido
import os
import plotly.graph_objects as go
import locale
from decimal import Decimal

In [2]:
path = os.getcwd()

benchmarks = {}

with os.scandir(path) as fileNames:
    for file in fileNames:
        if file.is_file():
            if file.name.endswith(".csv"):
                print(file.name)
                stats = file.name.split(".")
                instance = "{}.{}".format(stats[0], stats[1])
                benchmarks[instance] = file.name
                #benchmarks.append(stats[0] + "." + stats[1])

print(benchmarks)

c6a.8xlarge.1m-m16-16d-200ef.csv
c4.4xlarge.1m-m16-16d-200ef.csv
c7g.8xlarge.1m-m16-16d-200ef.csv
c6g.8xlarge.1m-m16-16d-200ef.csv
c6i.8xlarge.1m-m16-16d-200ef.csv
c6i.4xlarge.1m-m16-16d-200ef.csv
c5.4xlarge.1m-m16-16d-200ef.csv
{'c6a.8xlarge': 'c6a.8xlarge.1m-m16-16d-200ef.csv', 'c4.4xlarge': 'c4.4xlarge.1m-m16-16d-200ef.csv', 'c7g.8xlarge': 'c7g.8xlarge.1m-m16-16d-200ef.csv', 'c6g.8xlarge': 'c6g.8xlarge.1m-m16-16d-200ef.csv', 'c6i.8xlarge': 'c6i.8xlarge.1m-m16-16d-200ef.csv', 'c6i.4xlarge': 'c6i.4xlarge.1m-m16-16d-200ef.csv', 'c5.4xlarge': 'c5.4xlarge.1m-m16-16d-200ef.csv'}


In [3]:
#filename = "c4.4xlarge.1m-m16-16d-200ef.csv"
bench = pd.DataFrame()
for instance in benchmarks:
    print(instance)
    bh = pd.read_csv(benchmarks[instance])
    bh["instance"] = instance
    # Concatenate all DataFrames
    bench = pd.concat([bench, bh], ignore_index=True)

#bh.head()
bench.head(4)

c6a.8xlarge
c4.4xlarge
c7g.8xlarge
c6g.8xlarge
c6i.8xlarge
c6i.4xlarge
c5.4xlarge


,Dim,Size,K,M,Mmax,Mmax0,Ef,EfSearch,Heuristic,CpuType,...,BruteSearchMulti,BruteSearchSingle,HNSWSearchSecs,HNSWSearchMulti,HNSWSearchSingle,GroundTruthHits,HNSWPrecision,DateStart,DateEnd,instance
0,16,1000000,10,16,16,32,200,10,True,AMD EPYC 7R13 Processor,...,767.504476,23.984515,23.219111,43067.971137,1345.874098,7841655,0.784165,2023-08-16 23:33:13.933183479 +0000 UTC m=+0.0...,2023-08-16 23:56:49.534409073 +0000 UTC m=+141...,c6a.8xlarge
1,16,1000000,10,16,16,32,200,20,True,AMD EPYC 7R13 Processor,...,767.504476,23.984515,22.387781,44667.222251,1395.850695,8698702,0.869870,2023-08-16 23:33:13.933183479 +0000 UTC m=+0.0...,2023-08-16 23:57:12.817534862 +0000 UTC m=+143...,c6a.8xlarge
2,16,1000000,10,16,16,32,200,30,True,AMD EPYC 7R13 Processor,...,767.504476,23.984515,23.060493,43364.207177,1355.131474,9103201,0.910320,2023-08-16 23:33:13.933183479 +0000 UTC m=+0.0...,2023-08-16 23:57:36.768610727 +0000 UTC m=+146...,c6a.8xlarge
3,16,1000000,10,16,16,32,200,40,True,AMD EPYC 7R13 Processor,...,767.504476,23.984515,26.129559,38270.833134,1195.963535,9359847,0.935985,2023-08-16 23:33:13.933183479 +0000 UTC m=+0.0...,2023-08-16 23:58:03.820121329 +0000 UTC m=+148...,c6a.8xlarge


In [4]:
bench.shape

(140, 28)

In [5]:
bench["bruteqps"] = bench.apply(lambda row: (row["Size"] / row["BruteSearchSecs"]), axis=1)
#bench["bruteqps"]

In [6]:
bench["hnswqps"] = bench.apply(lambda row: (row["Size"] / row["HNSWSearchSecs"]), axis=1)
#bench["hnswqps"]

In [7]:
bench["hnswvsbrutesearch"] = bench.apply(lambda row: (row["hnswqps"] / row["bruteqps"]), axis=1)
#bench["hnswvsbrutesearch"]

In [16]:
#bench_pv = bench.pivot_table(index = 'HNSWPrecision', columns='HNSWSearchSecs', values='HNSWPrecision')
locale.setlocale(locale.LC_ALL, 'en_us')

plotLables={
    "hnswqps": "HNSW Search (queries p/second)",
    "HNSWPrecision": "HNSW Precision (%)",
}

plotTitle = "HNSW Search Time for {:n} vectors ({} dim, M {})".format(Decimal(int(bench["Size"][0])), bench["Dim"][0], bench["M"][0])
fig = plx.line(bench, x="HNSWPrecision", y="hnswqps", color="instance", title=plotTitle, labels=plotLables)

fig.update_xaxes(categoryorder='total descending')

fig.write_image("hnsw_search_all_instances.png", scale=4)
fig.write_html("hnsw_search_all_instances.html")
fig.show()

In [17]:
plotTitle = "Brute Search QPS for {:n} vectors ({} dim)".format(Decimal(int(bench["Size"][0])), bench["Dim"][0], bench["M"][0])
plotLables={
    "bruteqps": "Brute Search Time (queries p/second)",
    "instance": "Instance Type",
}


fig = plx.bar(bench.query("EfSearch == 10"), x="instance", y="bruteqps", color="instance", title=plotTitle, labels=plotLables)
fig.update_xaxes(categoryorder='total descending')

fig.write_image("brute_search_all_instances.png", scale=4)
fig.write_html("brute_search_all_instances.html")
fig.show()

In [18]:
plotTitle = "HNSW vs Brute Search Speedup for {:n} vectors ({} dim, M {})".format(Decimal(int(bench["Size"][0])), bench["Dim"][0], bench["M"][0])
plotLables={
    "hnswvsbrutesearch": "Speedup over Brute search (x times)",
    "HNSWPrecision": "HNSW Precision (%)",
    "instance": "Instance",
}

fig = plx.line(bench, x="HNSWPrecision", y="hnswvsbrutesearch", color="instance", title=plotTitle, labels=plotLables)

fig.update_xaxes(categoryorder='total descending')

fig.write_image("hnsw_vs_brute_search_speedup.png", scale=4)
fig.write_html("hnsw_vs_brute_search_speedup.html")
fig.show()

In [9]:
# TODO: Compare costs between each instance for price/performance ratio
#import boto3
#import json
#import pprint

#pricing = boto3.client('pricing', region_name='us-east-1')

#ec2_price = {}
#boto3.client('sts').get_caller_identity().get('Account')

#for inst in bench["instance"].unique():

#    response = pricing.get_products(
#        ServiceCode='AmazonEC2',
#        Filters = [
#            {'Type' :'TERM_MATCH', 'Field':'instanceType', 'Value':inst},
#            {'Type' :'TERM_MATCH', 'Field':'operatingSystem', 'Value':'Linux'},
#            { 'Type': 'TERM_MATCH', 'Field': 'capacitystatus', 'Value': 'Used' },
#            { 'Type': 'TERM_MATCH', 'Field': 'preInstalledSw', 'Value': 'NA' },
#            { 'Type': 'TERM_MATCH', 'Field': 'tenancy', 'Value': 'shared' },
#            {'Type' :'TERM_MATCH', 'Field':'regionCode',        'Value':'us-east-1'}
#        ],
#        MaxResults=1
#    )

#    ec2_price[inst] = json.loads(response['PriceList'][0])

In [10]:
#ec2_per_hour = {}

#for inst in bench["instance"].unique():
#    for price in ec2_price[inst]["terms"]["OnDemand"]:
#        for dim in ec2_price[inst]["terms"]["OnDemand"][price]["priceDimensions"]:
#            print(inst, ec2_price[inst]["terms"]["OnDemand"][price]["priceDimensions"][dim]["pricePerUnit"]["USD"])
#            ec2_per_hour[inst] = ec2_price[inst]["terms"]["OnDemand"][price]["priceDimensions"][dim]["pricePerUnit"]["USD"]


In [11]:
#bench["ec2perhour"] = bench.apply(lambda row: (float(ec2_per_hour[row["instance"]])), axis=1)
#bench["cost"] = bench.apply(lambda row: ( ((float(row["HNSWSearchSecs"])*60)/bench["ec2perhour"])), axis=1)

In [12]:
#bench.head(5)

In [13]:
#plotTitle = "Cost for QPS for {:n} vectors ({} dim)".format(Decimal(int(bench["Size"][0])), bench["Dim"][0], bench["M"][0])
#plotLables={
#    "bruteqps": "Brute Search Time (queries p/second)",
#    "instance": "Instance Type",
#}

#fig = plx.bar(bench.query("EfSearch == 100"), x="instance", y="ec2perhour", color="instance", title=plotTitle, labels=plotLables)
#fig.update_xaxes(categoryorder='total descending')

#fig.write_image(filename.replace(".csv", ".png"), scale=4)
#fig.write_html(filename.replace(".csv", ".html"))
#fig.show()

In [14]:
#ops_pv = bench.pivot_table(index = 'instance', columns='ec2perhour', values='cost')

In [15]:
#ops_pv